Here we will take in a print document such as a PDF or an EPUB. We will then create embeddings for it and use an LLM to answer questions about it.

In [36]:
from tqdm import tqdm
import google.generativeai as palm
from IPython.display import display, Markdown

In [1]:
with open("data/book_chapter.txt", encoding='utf-8') as f:
    doc = f.read()

### Chunking

In [2]:
from chunker import get_chunks

In [6]:
# Make type suitable for chunker function
docs = []
docs.append(doc)

['The expansion of Europe during the last century has been the story of\ncrime and violence against backward peoples under the cloak of\nprotective civilisation.\n—CAPTAIN RICHARD MEINERTZHAGEN1\nLIKE SO MANY OTHER AREAS OF THE BRITISH EMPIRE, EAST AFRICA\nwas opened to colonial domination with the building of railroads, the\nsymbol of imperial achievement. By the early twentieth century\nthousands of miles of tracks crisscrossed Africa and Asia, opening\nthe territories to the forces of Pax Britannica. Beginning in August\n1896, the British government financed and directed the laying of 582\nmiles of track stretching from the coastal port of Mombasa all the\nway to Lake Victoria and beyond. The line cut through the lush and\nexotic landscape of the Indian Ocean coast, then through the arid,\nlion-infested plains of Tsavo, and finally upcountry toward the Eden\nof the interior highlands. Completed in December 1901, it was called\nthe Uganda Railway for it linked the inland territory of

In [23]:
texts = get_chunks(docs)
texts

['The expansion of Europe during the last century has been the story of\ncrime and violence against backward peoples under the cloak of\nprotective civilisation.\n—CAPTAIN RICHARD MEINERTZHAGEN1\nLIKE SO MANY OTHER AREAS OF THE BRITISH EMPIRE, EAST AFRICA\nwas opened to colonial domination with the building of railroads, the\nsymbol of imperial achievement. By the early twentieth century\nthousands of miles of tracks crisscrossed Africa and Asia, opening\nthe territories to the forces of Pax Britannica. Beginning in August\n1896, the British government financed and directed the laying of 582\nmiles of track stretching from the coastal port of Mombasa all the\nway to Lake Victoria and beyond. The line cut through the lush and\nexotic landscape of the Indian Ocean coast, then through the arid,\nlion-infested plains of Tsavo, and finally upcountry toward the Eden\nof the interior highlands. Completed in December 1901, it was called\nthe Uganda Railway for it linked the inland territory of

### Embeddings

In [24]:
from uuid import uuid4
from embeddings_palm import get_palm_embeddings

In [25]:
chunks = []
for text in tqdm(texts):
    
    chunks.append(
        {
            'id': str(uuid4()),
            'values': get_palm_embeddings(text),
            'metadata': {
                'text': text
                }
        }
    )

100%|██████████| 44/44 [02:25<00:00,  3.31s/it]


## Pinecone

In [26]:
import os
import pinecone

### Credentials

In [27]:
pinecone_api_key = os.getenv('PINECONE_API_KEY')

### Creating an Index

In [28]:

index_name = 'general-rag-idx'

# initialize connection to pinecone
pinecone.init(
    api_key= pinecone_api_key,
    environment="gcp-starter"  # next to API key in console
)

In [29]:
# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=len(chunks[0]['values']),
        metric='dotproduct'
    )
# connect to index
index = pinecone.GRPCIndex(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.00101,
 'namespaces': {'': {'vector_count': 101}},
 'total_vector_count': 101}

### Populating the Index

In [30]:
index.upsert(vectors=chunks)

upserted_count: 44

### Retrieval

#### Create an Embedding for the Query
This comes in the form of a query vector we will name `xq`.

In [31]:
query = "How did the British introduce the wage labour system in kenya?"

xq = get_palm_embeddings(query)


#### Query Vector Database

In [32]:
res = index.query(xq, top_k=5, include_metadata=True)
res

{'matches': [{'id': '373796d8-79ff-46b4-bdbe-84602ee64afa',
              'metadata': {'text': 'had the fundamental role of maintaining '
                                   '“tribal order.”34\n'
                                   'But the Kikuyu did not have chiefs. Prior '
                                   'to colonialism, they were\n'
                                   'a stateless society, governed by councils '
                                   'of elders and lineage heads.\n'
                                   'In Kikuyu districts these new chiefs were '
                                   'a phenomenon of colonial\n'
                                   'rule. They were created by the colonial '
                                   'government and thus wholly\n'
                                   'illegitimate in the eyes of ordinary '
                                   'Kikuyu people. By accepting\n'
                                   'British authority, the chiefs were granted '
 

## Retreival Augmented Generation

### Stuff Method
We concatenate the text metadata from the vector store directly with our query. This is simple and it works for small prompts.

In [33]:
# get list of retrieved text
contexts = [item['metadata']['text'] for item in res['matches']]

augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query

### Primer Template

In [34]:
# system message to 'prime' the model
primer = f"""
You are a researcher doing historical and anthropological work on british colonialism in east africa
"""

In [39]:
res = palm.chat(context=primer, messages=augmented_query, temperature=0.7)

In [40]:
display(Markdown(res.last))

The British introduced a wage labor system in Kenya through a series of policies and regulations. These policies included:

* **Confinement of Africans to reserves.** The British colonial government confined Africans to reserves, which were areas of land set aside for African use. This policy was designed to limit African access to land and resources, and to make them more dependent on wage labor.
* **Taxation.** The British colonial government imposed taxes on Africans, which they had to pay in cash. This policy forced Africans to find wage labor in order to earn the money to pay their taxes.
* **Pass laws.** The British colonial government required Africans to carry passes, which were documents that recorded their identity and employment status. These passes were used to control the movement of Africans and to prevent them from leaving the reserves without permission.
* **Labor recruitment.** The British colonial government recruited Africans to work on European farms and plantations. This recruitment was often done through coercion, and Africans were often forced to work in dangerous and unhealthy conditions.

The introduction of the wage labor system had a devastating impact on the lives of Africans in Kenya. It led to the breakdown of traditional African society, as people were forced to leave their homes and families to work for low wages. It also led to the impoverishment of many Africans, as they were unable to earn enough money to support themselves and their families.

The wage labor system also led to a great deal of resentment among Africans. They felt that they were being exploited by the British, and that they were not being given a fair share of the profits from their labor. This resentment eventually led to the Mau Mau Uprising, a rebellion against British rule.